In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.datasets import fetch_openml
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.tree import DecisionTreeClassifier

data = fetch_openml('mnist_784', version=1)#Get data from https://www.openml.org/d/554
dfData = pd.DataFrame(np.c_[data["data"],data["target"]],columns = data["feature_names"]+["target"])

In [2]:
stratSplit = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=0)

for train_index, test_index in stratSplit.split(dfData[data["feature_names"]], dfData["target"]):
    X_train = dfData[data["feature_names"]].iloc[train_index]
    X_test = dfData[data["feature_names"]].iloc[test_index]
    
    y_train = dfData["target"].iloc[train_index]
    y_test = dfData["target"].iloc[test_index]

In [3]:
from sklearn.ensemble import BaggingClassifier

#soft voting by default if predict_proba method exists
bag_clf = BaggingClassifier(base_estimator=DecisionTreeClassifier(max_depth=3,
                                                      max_features='sqrt',
                                                      splitter='random'),
                            n_estimators=30,
                            max_samples=0.8,  # int/float, default with replacement
                                              # maximum training set sample size compared to original training set 
                            max_features=0.9, # int/float, default without replacement
                                              # maximum feature size compared to original number of features 
                            bootstrap=True,   # bagging (True)/pasting (False) (affects max_samples behaviour)
                            bootstrap_features=False, # if feature selection should use baggin (max_features)
                            oob_score=False,  # perform oob scoring
                            warm_start=False,
                            n_jobs=2,
                            random_state=0,
                            verbose=0
                            )

bag_clf.fit(X_train, y_train)

BaggingClassifier(base_estimator=DecisionTreeClassifier(ccp_alpha=0.0,
                                                        class_weight=None,
                                                        criterion='gini',
                                                        max_depth=3,
                                                        max_features='sqrt',
                                                        max_leaf_nodes=None,
                                                        min_impurity_decrease=0.0,
                                                        min_impurity_split=None,
                                                        min_samples_leaf=1,
                                                        min_samples_split=2,
                                                        min_weight_fraction_leaf=0.0,
                                                        presort='deprecated',
                                                        random_state=None,
 

In [4]:
print(len(bag_clf.estimators_)) # array of trained estimators
print(len(bag_clf.estimators_samples_)) # array of sample subsets for each estimator
print(len(bag_clf.estimators_features_)) # array of feature subsets for each estimator

# #when oob_score is true
# bag_clf.oob_score_
# bag_clf.oob_decision_function_

30
30
30


In [5]:
scores = []
for est,features in zip(bag_clf.estimators_,bag_clf.estimators_features_):
    scores.append(est.score(X_test.values[:, features], y_test.astype(np.float64)))
    # Sklearn transform str for of integers to np.float64 internally

In [6]:
print('Avg. estimator performance:',np.mean(scores))
print('Estimaor performance std. dev.:',np.std(scores))

Avg. estimator performance: 0.3655285714285715
Estimaor performance std. dev.: 0.019923472636685657


In [7]:
bag_clf.score(X_test, y_test)

0.7021428571428572